In [1]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F

In [2]:
#下载载入数据集
dataset_train = torchvision.datasets.MNIST(root='./../all_dataset',train=True,transform=torchvision.transforms.ToTensor(),download=False)
dataset_test = torchvision.datasets.MNIST(root='./../all_dataset',train=False,transform=torchvision.transforms.ToTensor(),download=False)

dataloader_train = torch.utils.data.DataLoader(dataset=dataset_train,batch_size=100,shuffle=True)
dataloader_test = torch.utils.data.DataLoader(dataset=dataset_test,batch_size=400,shuffle=False)

In [3]:
#模型定义
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.fc1 = nn.Linear(28*28,100)
        self.fc2 = nn.Linear(100,10)
        
    def forward(self,x):
        x = F.sigmoid(50*self.fc1(x))
        x = self.fc2(x)
        return x
    
model = MLP()    
#定义损失函数与优化器
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum = 0.9)

In [ ]:
model = model.cuda()
loss_func = loss_func.cuda()

optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum = 0.9)
epochs = 50
accuracy_list = []
for epoch in range(epochs):
    sum_loss = 0
    for i, (images, labels) in enumerate(dataloader_train):
        images = images.reshape(images.shape[0],-1).cuda()
        labels = labels.cuda()
        pred = model(images)
        loss = loss_func(pred,labels)
        
        optimizer.zero_grad()
        loss.backward()            #此处loss是100个case的平均
        optimizer.step()
        sum_loss += loss

        if i%100 == 99:
            print('epoch:{},batch:{},train loss:{}'.format(epoch,i,sum_loss/100))
            sum_loss = 0

    correct = 0    
    for j, (images, labels) in enumerate(dataloader_test):
        images = images.reshape(images.shape[0],-1).cuda()
        labels = labels.cuda()
        pred = model(images)
        values,indices = torch.max(pred,1)
        correct += (indices==labels).sum()
        
    print('epoch:{},test accuracy:{}'.format(epoch,correct/10000))
    
    accuracy_list.append(correct/10000)
print(accuracy_list)

epoch:0,batch:99,train loss:0.026735324412584305
epoch:0,batch:199,train loss:0.019176336005330086
epoch:0,batch:299,train loss:0.02272714488208294
epoch:0,batch:399,train loss:0.02317882515490055
epoch:0,batch:499,train loss:0.023852279409766197
epoch:0,batch:599,train loss:0.020715082064270973
epoch:0,test accuracy:0.9685999751091003
epoch:1,batch:99,train loss:0.01897362247109413
epoch:1,batch:199,train loss:0.022110020741820335
epoch:1,batch:299,train loss:0.0248994342982769
epoch:1,batch:399,train loss:0.024015776813030243
epoch:1,batch:499,train loss:0.025390760973095894
epoch:1,batch:599,train loss:0.02091223932802677
epoch:1,test accuracy:0.9681999683380127
epoch:2,batch:99,train loss:0.019071480259299278
epoch:2,batch:199,train loss:0.028251972049474716
epoch:2,batch:299,train loss:0.02171424962580204
epoch:2,batch:399,train loss:0.023220812901854515
epoch:2,batch:499,train loss:0.02070779912173748
epoch:2,batch:599,train loss:0.02329498901963234
epoch:2,test accuracy:0.968299

In [26]:
model = model.cpu()
torch.save(model.state_dict(),'./MLP784_100_10_1sig0.2x_9758.pt')

In [11]:
model_param = torch.load('./MLP784_100_10_sig0.2x_9758.pt')
for param in model_param:
    print(param)
    print(model_param[param].shape)
    print(model_param[param].abs().max())

fc1_factor = torch.max(model_param['fc1.weight'].abs().max(),model_param['fc1.bias'].abs().max())
fc2_factor = torch.max(model_param['fc2.weight'].abs().max(),model_param['fc2.bias'].abs().max())
w_factor = torch.max(fc1_factor,fc2_factor)
print(fc1_factor,fc2_factor)

w_bit = 8
h_level = 2**(w_bit-1)-1
delta_w = 1/h_level
model_param['fc1.weight'] = (model_param['fc1.weight']/w_factor*h_level).round()*delta_w
model_param['fc1.bias'] = (model_param['fc1.bias']/w_factor*h_level).round()*delta_w
model_param['fc2.weight'] = (model_param['fc2.weight']/w_factor*h_level).round()*delta_w
model_param['fc2.bias'] = (model_param['fc2.bias']/w_factor*h_level).round()*delta_w

fc1.weight
torch.Size([100, 784])
tensor(3.0247)
fc1.bias
torch.Size([100])
tensor(2.8674)
fc2.weight
torch.Size([10, 100])
tensor(8.4427)
fc2.bias
torch.Size([10])
tensor(0.2712)
tensor(3.0247) tensor(8.4427)


In [12]:
torch.save(model.state_dict(),'./MLP784_100_10q8_sig0.2x.pt')

In [13]:
#量化网络
model_param = torch.load('./MLP784_100_10q8_sig0.2x.pt')
model.load_state_dict(model_param)

correct = 0    
for j, (images, labels) in enumerate(dataloader_test):
    images = images.reshape(images.shape[0],-1)
    images = (images)
    pred = model(images)
    values,indices = torch.max(pred,1)
    correct += (indices==labels).sum()
print(correct)

tensor(3503)


In [5]:
import numpy as np
count = 0
for j, (images, labels) in enumerate(dataloader_test):
    print(images.shape)
    images = images.reshape(images.shape[0],-1)
    result = np.where(images == 0,1,0)
    count += result.sum()

print(count)

torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
torch.Size([400, 1, 28, 28])
6328781
